In [1]:
## selenium class for Farmacias Especializadas
##This code Scrapes Actual product description, Barcodes and Manufacturer Names using product description from dataset and generates a similarity score for the scraped product

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

class TestSecondtest():
    def setup_method(self, method):   #it is suggested for mac users to download chrome drivers for selenium and mention path.
        self.driver = webdriver.Chrome()

        self.vars = {}


    def teardown_method(self, method):
        self.driver.quit()

    def initialSetup(self):
        self.driver.get("https://www.farmaciasespecializadas.com/")
        self.driver.set_window_size(1440, 761)   


    def searchElement(self,name):
            searchBar = self.driver.find_element(By.NAME,"s")
            searchBar.click()
            searchBar.send_keys(name)
            searchBar.send_keys(Keys.ENTER)
            return True

    def clickTile(self,value):
        self.driver.find_element(By.CSS_SELECTOR, f".card__item:nth-child({value+1}) .image__item").click()

    def getDetails(self):
        try: 
            upc =self.driver.find_elements(By.CSS_SELECTOR, \
                                           "#cc-product-details > div > div > div.oc-panel.panel-2.col-md-6 > span")
            
            company = self.driver.find_elements(By.CSS_SELECTOR,\
                                            "#cc-product-details > div > div > div.oc-panel.panel-2.col-md-6 > div:nth-child(11) > p.text.text__little.text-base1.base2.product__detail--top.lab_name")
            return upc[0].text,company[0].text
        except:
            return "",""
        
    def findElementFromList(self):
        try:
            names =self.driver.find_elements(By.CSS_SELECTOR, "#gridProducts > div > div > div > a > p > span") 
            descriptions = self.driver.find_elements(By.CSS_SELECTOR, ".card__item .campos.ln2 span")
            fullnames = []
            count = 0
            temp = ""
            for i in names:
                if i is not None:
                    temp+= i.text+" "
                count +=1
                if (count %2 == 0):
                    fullnames.append(temp+descriptions.pop(0).text)
                    temp =""
                    
            return fullnames
        except Exception as e:
            print(e)
            return []




In [2]:
import unicodedata

## used to normalize data 
## like Â ,Ć ,Ê , to A , C , E

def normalize(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')

In [3]:
import re
def Tab_type_details(string):
    
    tablet_type_features = []
    ## most found tablets types are stored in below dictionary
    tab_types = {
                   "CREAM":["CREAM","CREMA","POMADA","OINTMENT"],
                   "DROPS":["GTS","GOTAS"],
                   "BOTTLE":["FRESCO","FRASCO","FRASCO AMPOLLA","SOL"],
                   "TABLET":["TABLETAS","TABLETS","TABLET","TABS","TAB","GRAGEAS"],
                   "POWDER" :["SOBRES","SOBRE"],
                   "INJECTION":["INY","I[.]V.","I[.]V","I V","IV","AMP","AMPOLLETAS","AMPOLLA","AMPULA",\
                                "VIAL","VIALS","JER","FA","F[.]A","F A","F[.]A[.]"],
                   "SUPO":["SUPOSITORIS","SUPPOSITORIES","SUPOSITORIO","SUPPOSITORIES"],
                   "CAPSULE":["COMPRIMIDOS","COMPR","COPR","COMP","CPR","CAPSULES","CAPSULAS","CAPS","CAP"]
                  }
    
    ## Extracting tablet type and count info combinely that have con,c/,tabcount "
    for tab_type in tab_types.keys():        
        for name in tab_types[tab_type]:
            found = re.findall(f"C?O?N?/? ?\d+(?:\.\d+)? ?{name}.? ?", string)
            for i in found:
                count = re.findall("(\d+(?:\.\d+)?)",i)[0]
                tablet_type_features.append({"raw_string":i,"count":count,"found_name":name,"type":tab_type})
                string = string.replace(i," ")
    return tablet_type_features,string
    

                
            

In [4]:
## some more fine tuning:

def extractmorefeatures(string):
    extra_features = {"types":[],"counts":[],"common":[]}
    tab_types = {
                   "CREAM":["CREAM","CREMA","POMADA","OINTMENT"],
                   "DROPS":["GTS","GOTAS"],
                   "BOTTLE":["FRESCO","FRASCO","FRASCO AMPOLLA","SOL"],
                   "TABLET":["TABLETAS","TABLETS","TABLET","TABS","TAB","GRAGEAS"],
                   "POWDER" :["SOBRES","SOBRE"],
                   "INJECTION":["INY","I[.]V.","I[.]V","I V","IV","AMP","AMPOLLETAS","AMPOLLA","AMPULA",\
                                "VIAL","VIALS","JER","FA","F[.]A","F A","F[.]A[.]"],
                   "SUPO":["SUPOSITORIS","SUPPOSITORIES","SUPOSITORIO","SUPPOSITORIES"],
                   "CAPSULE":["COMPRIMIDOS","COMPR","COPR","COMP","CPR","CAPSULES","CAPSULAS","CAPS","CAP"]
                  }
    COMMON_TERMS = ["CAJA","CON","PISA"]

    ## to check for types
    for tab_type in tab_types.keys():        
        for name in tab_types[tab_type]:
            found = re.findall(f" {name} ", string)
            if not found:
                found = re.findall(f" {name}\Z",string)
            for i in found:
                extra_features["types"].append({"raw_string":i,"found_name":name,"type":tab_type})
                string = string.replace(i," ")
   
            
    # to check for attributes like tabtype and tabcount individually
    
    found = re.findall("C?O?N?/? ?\d+(?:\.\d+)?",string)
    for i in found:
        string = string.replace(i," ")
        count = re.findall("(\d+(?:\.\d+)?)",i)[0]
        extra_features["counts"].append({"raw_string":i,"count":count})
    
    # extracting some common terms and extra info from the tablet
    for name in COMMON_TERMS:
        found = re.findall(f" {name} ", string)

        if not found:
            found = re.findall(f" {name}\Z",string)
        for i in found:
            extra_features["common"].append(i)
            string = string.replace(i," ")
    return extra_features,string
# print(cleanstring("NEO-MELUBRINA INFANTIL  /  FRASCO CON "))
    
    
    

In [5]:
import re

# This Function extracts weight ,composition,volumes info from the tablet
def findCWV(string):

    COMPOSITIONS = ["MG/ML","UI/ML","IU/ML","G/ML","U[.]I/ML","U[.]I/ML","U/ML","%","U[.]I","UI"]
    VOLUMES = ["ML","L"]
    WEIGHTS = ['GRAMS', 'GRM', 'GRS', 'GR', 'G', 'MCG', 'MC', 'MG',]
    CWV =[[],[],[]]
    for num,QUALITIES in enumerate([COMPOSITIONS,WEIGHTS,VOLUMES]):   
        for i in QUALITIES:
            ##find multiple and replace 
            regex = re.compile(f'\d+(?:\.\d+)? ?/ ?\d+(?:\.\d+)? ?{i}| ?\d+(?:\.\d+)? ?/\d+(?:\.\d+)? ?/\d+(?:\.\d+)? ?{i}')
            matches = re.findall(regex, string)

            for match in matches:
                final_string = ""

                temp = match.split("/")
                for t in temp:
                    final_string += t+f"{i} " if i not in t else t

                string = re.sub(match,final_string,string)

            
#             print(string)
            #
            temp = re.findall(f"\d+(?:\.\d+)? ?{i}[\),., ,\/]", string)
            for j in temp:
                CWV[num].append(j)
                string = string.replace(j," ")

    return CWV , string

    
findCWV("EXFORGE HCT 320/25G ")

([[], ['320G ', '25G '], []], 'EXFORGE HCT   ')

In [6]:
import re

# Extracts the tablet name after removing all important qualities

def findName(string,CWV,tab_types,extrafeatures):

    raw_strings = []
    CWV.append(["("])

    for i in tab_types:
        CWV[-1].append(i["raw_string"])
    for feature in [extrafeatures["types"],extrafeatures["counts"]]:
        for i in feature:
            CWV[-1].append(i["raw_string"])
    min_pos_found = len(string)
    for qualities in CWV:
        for quality in qualities:
            if (quality != ''):

                position = string.find(quality)
                if position != -1:
                    min_pos_found = min(min_pos_found,position)
    return cleanstring(string[:min_pos_found])

            

In [7]:
import re

def cleanstring(text):

    """
        This function removes characters \,/,*,-,. 

    """
    text = re.sub("[\),.,\-,*,/,\(]"," ",text)
    text = text.strip()
    return text
# print(cleanstring("ASOFLON 0.4 MG C/30 CAP ( TAMSULO-SINA )wejnfe "))

In [8]:
import re

def standardize(string):
    
    """
        This function replaces all patterns like GRAMS|GRM|GRS|GR with G

    """
    # Define the regular expression pattern
    pattern = re.compile(r'GRAMS|GRM|GRS|GR')

    # Replace the substrings with 'g'
    output_string = re.sub(pattern, 'G', string)
    output_string = re.sub("[\),\ ,\(,\/]","",output_string)
    output_string = output_string.strip(".")

    # Print the output string
    return output_string

standardize("10/ 20/25 MG.")        

'102025MG'

In [9]:
def Extract_Details(text):

    """
        This Function takes whole tablet description as input 
        and returns compositions, weights and volumes,name,features.    

    """
    text = (" ".join(text.split())).strip()
    text = text +" "
    text = normalize(text).upper()
    string = normalize(text).upper()
    details = {}
    length = len(string)
    CWV,string = findCWV(string)
    for key, i in enumerate(["compositions","weights","volumes"]):
        details[i]=CWV[key]
    details["tab_type_details"],string= Tab_type_details(string)
    extrafeatures,string = extractmorefeatures(string)
    details["name"] = findName(text,CWV,details["tab_type_details"],extrafeatures)
    string = string.replace(details["name"],"")
    string = (" ".join(string.split())).strip()
    string = cleanstring(string)
    details["extrafeatures"] = extrafeatures
    for quant in ["weights","volumes","compositions"]:
        for key,i in enumerate(details[quant]):
            details[quant][key] = standardize(i)
    types = set()
    counts = set()            
    for detail in details["tab_type_details"]:
        if len(details["tab_type_details"]) != 0:
            types.add(detail["type"])
            counts.add(detail["count"])
    for detail in details['extrafeatures']['types']:
        types.add(detail["type"])
    for detail in details['extrafeatures']['counts']:
        counts.add(detail["count"])
    details["features"] = {"types":types,"counts":counts,"extra":set(string.split())}
    details.pop('tab_type_details')
    details.pop('extrafeatures')    
    return details

Extract_Details("NIQUITIN 21 MG C/7 PARCHES CON PARA")
Extract_Details("EXFORGE HCT 10MG/320MG/25MG CON 28 COMPRIMIDOS TAB")
Extract_Details("SIMPLE 50 ML")

{'compositions': [],
 'weights': [],
 'volumes': ['50ML'],
 'name': 'SIMPLE',
 'features': {'types': set(), 'counts': set(), 'extra': set()}}

In [10]:
def similarity_type(target_type,src_type):
    
    """
        This Function considers correlation between two types to 
        measure similarity between two types and return output as 
        value between 0 and 1

    """
    similarity_dict ={
        "SUPOCREAM" : 0.24,
        "SUPODROPS" : 0.3,
        "SUPOBOTTLE" : 0.24,
        "SUPOTABLET" : 0.35,
        "SUPOPOWDER" : 0.29,
        "SUPOINJECTION" : 0.2,
        "SUPOCAPSULE" : 0.39,
        "CREAMDROPS" : 0.59684736728668213,
        "CREAMBOTTLE" : 0.5013158917427063,
        "CREAMTABLET" : 0.33642226457595825,
        "CREAMPOWDER" : 0.48533108830451965,
        "CREAMINJECTION" : 0.49305346608161926,
        "CREAMCAPSULE" : 0.3910931646823883,
        "DROPSBOTTLE" : 0.7995974540710449,
        "DROPSTABLET" : 0.603466272354126,
        "DROPSPOWDER" : 0.583532989025116,
        "DROPSINJECTION" : 0.6436945199966431,
        "DROPSCAPSULE" : 0.6268869638442993,
        "BOTTLETABLET" : 0.5191242694854736,
        "BOTTLEPOWDER" : 0.6567681431770325,
        "BOTTLEINJECTION" : 0.92448949813843,
        "BOTTLECAPSULE" : 0.5893133878707886,
        "TABLETPOWDER" : 0.5272809267044067,
        "TABLETINJECTION" : 0.5583986043930054,
        "TABLETCAPSULE" : 0.92078971862793,
        "POWDERINJECTION" : 0.6065213680267334,
        "POWDERCAPSULE" : 0.5563412308692932,
        "INJECTIONCAPSULE" : 0.6149609088897705
    }
    target_type = target_type.strip()
    src_type = src_type.strip()
#     print(f'{target_type}{src_type}')
    if(f'{target_type}{src_type}' in similarity_dict):
        return similarity_dict[f"{target_type}{src_type}"]
    if(f'{src_type}{target_type}' in similarity_dict):
        return similarity_dict[f"{src_type}{target_type}"]
        
    return 0


# similarity("TABLET","CREAM")

In [11]:
import jellyfish
def nameScore(s1, s2):

    """
        This function measures similarity between two tablet names and
        gives output as normalized value between 0 and 1
    """
    tokens1 = set(s1.split())
    tokens2 = set(s2.split())
    jaccard_sim = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))
    levenshtein_dist = jellyfish.levenshtein_distance(s1, s2)
    similarity = min(0.8*jaccard_sim + 0.2* (2 - (levenshtein_dist / max(len(s1), len(s2)))),1 )

    return similarity


In [12]:
def surityscore(src,target):
    
    """
        This Function measures score using parameters like weight,volume,
        composition,name,extrafeatures and calculates surity score between
        two strings
    """
    src = Extract_Details(src)
    target = Extract_Details(target)
    score = 0
    maxscore = 0
    # Weightages for different types 
    weightages ={
        "name":3000,
        "weights":700,
        "volumes":700,
        "compositions":200,
        "types":1000,
        "counts":1000,
        "extra":70,
    }

    maxscore+=weightages["name"] 
    namescore = nameScore(src["name"] , target["name"])
    score +=namescore * weightages["name"]
    

    ## calculate weight,volume,compositions score
    for quantity in ["weights","volumes","compositions"]:
        if len(src[quantity]) != 0 or len(target[quantity]) !=0:

            for weight in src[quantity]:
                maxscore+= weightages[quantity]
                if weight in target[quantity]:  score += weightages[quantity]
                else: 

                    score -= weightages[quantity]
            for weight in target[quantity]:
                maxscore+= weightages[quantity]
                if weight in src[quantity]:  score += weightages[quantity]
                else: score -= weightages[quantity]
    ## calculate types,counts score
    for params in ["types","counts"]:
        
        if (len(src["features"][params]) == 0  and len(target["features"][params] ) !=0) or \
           (len(src["features"][params]) != 0 and len(target["features"][params] ) ==0) :
            maxscore+= weightages[params]
        elif (len(src["features"][params]) != 0 and len(target["features"][params] )) !=0:
            maxscore+= weightages[params] * (len(src["features"][params]) + len(target["features"][params]))
            for src_type in src["features"][params]:
                if src_type in target["features"][params]:
                    score+= weightages[params]
                else: 
                    score+= max([similarity_type(src_type,i) for i in target["features"][params]]) * weightages[params]
            for target_type in target["features"][params]:
                if target_type in src["features"][params]:
                    score+= weightages[params]
                else: 
                    score+= max([similarity_type(target_type,i) for i in src["features"][params]]) * weightages[params]

    ## calculate extra features score              
    for i in src["features"]["extra"]:
        maxscore += weightages["extra"]
        if i in target["features"]["extra"]: score +=weightages["extra"]
    for i in target["features"]["extra"]:
        maxscore += weightages["extra"]
        if i in src["features"]["extra"]: score +=weightages["extra"]       
    return score * 100/maxscore


    

In [13]:
import pandas as pd
# *Note: Due to recent update of this website, we need to close the pop up manually once the browser opens up.
# *Note: The output is printed in a CSV file as asked initially. But we would suggest to print it in a different format so that you won't face any issues with CSV modifying the barcodes/scraped data.

inputfile = "input.csv"    #use the product description column as input (output of DataCleaning is suggested)
outfile = open("output.csv",'a')
startline = 1   # starting row in excel for scraping
refresh = 1  #refreshes page after ever mentioned search to avoid errors caused due to website
required_similarity = 63   # minimum criteria for scraping a product

# Read CSV file into a Pandas DataFrame
df = pd.read_csv(inputfile,header=None)

# test initialization
test = TestSecondtest()
test.setup_method(None)
test.initialSetup()
test.driver.implicitly_wait(3)
time.sleep(20)  #time for the page to set up and load after opening the browser.
try:
    count = 0
    found = 0
    for element in df.iloc[startline - 1:,0]:   #[startline -1 : "if any specific last row in excel, can be mentioned here", 0]

        count +=1
        
        # try searching element, if not found try again by going to home page
        try:
            test.searchElement(element)
        except:
            try:
                print("search not found retrying going back  ...")
                test.driver.back()
                test.driver.refresh()
                test.searchElement(element)
            except:
                time.sleep(3)
                test.searchElement(element)
        
        fullnames = test.findElementFromList()
        similar =None
        print(f"Line {startline+count-1}: ",end = "")
        """
            if any items are found, find the item with
            maximum surity and checked if it is greater than
            required similarity. 
        """
        if fullnames:

            surity_indexes = [surityscore(element,target) for target in fullnames]
            max_surity_indexes = max(surity_indexes) if len(fullnames)>0 else 0
            if max_surity_indexes >= required_similarity:
                similar = surity_indexes.index(max_surity_indexes)
        """
            if any max surity element is found , 
            retrieve the company name , barcode ,
            fullname found and store in the output file in csv format
        """
        if similar!= None:
            test.clickTile(similar)
            time.sleep(1)
            upc,company = test.getDetails()
            found +=1
            fullnames[similar] = " ".join(fullnames[similar].split(","))
            company = " ".join(company.split(","))
            print(f"{normalize(element)},{normalize(fullnames[similar])},{upc},{normalize(company)},{max_surity_indexes}")
            outfile.write(f"{normalize(element)},{normalize(fullnames[similar])},{upc},{normalize(company)},{max_surity_indexes}\n")
        else:
            print(element,"Notfound")
            outfile.write(f"{normalize(element)}\n")
            
        if count % refresh == 0:
            print(f"\n ******** {found} found, out of {count} ********\n")
            test.driver.refresh()
    # close file and driver
    outfile.close()
    test.teardown_method(None)
except Exception as e:
    print(e)
    outfile.close()
    test.teardown_method(None)

Line 1: NEXIUM 40 MG TAB,NEXIUM - MUPS 40 MG Caja C/ 14 Tabletas,7501098611329,ASTRAZENECA,72.63823709447051

 ******** 1 found, out of 1 ********

Line 2: NEXIUM 40 MG. C 7 TAB,NEXIUM - MUPS 40 MG Caja C/ 7 Tabletas,7501098611312,ASTRAZENECA,87.55179246982524

 ******** 2 found, out of 2 ********

Line 3: NEXIUM IV 40 MG F A,NEXIUM IV 40 MG Solucion Inyectable Caja C/ 1 Frasco Ampula,5000456013666,ASTRAZENECA,85.76685595009872

 ******** 3 found, out of 3 ********

Line 4: NEXIUM MUPS 20 MG,NEXIUM - MUPS 20 MG Caja C/ 14 Tabletas,7501098611305,ASTRAZENECA,67.2782874617737

 ******** 4 found, out of 4 ********

Line 5: NEXIUM MUPS 20 MG 14 TABLETAS,NEXIUM - MUPS 20 MG Caja C/ 14 Tabletas,7501098611305,ASTRAZENECA,98.36065573770492

 ******** 5 found, out of 5 ********

Line 6: DUOVISC 0.55 ML Notfound

 ******** 5 found, out of 6 ********

Line 7: CONCENTRADO ACIDO DIALOG Notfound

 ******** 5 found, out of 7 ********

Line 8: DICHLOREX SOL 2% COLOREADA 125CC Notfound

 ******** 5 foun